In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import configparser
import os
import sys
import warnings
import wandb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_hub as hub
from albumentations import (
    Compose, 
    ShiftScaleRotate,
)
from sklearn.model_selection import (
    KFold,
    ShuffleSplit,
)
from wandb.keras import WandbCallback

sys.path.append("..")
from utils.common import (
    get_cpu_count,
    get_elapsed_time,
)
from utils.data_utils import (
    load_tfrecord_dataset,
    mixup_dataset,
)

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
warnings.filterwarnings("ignore")
np.random.seed(42)

In [ ]:
def view_image(dataset, n_samples, label_names=None):
    images, labels = next(iter(dataset))
    images = images.numpy()
    labels = labels.numpy()

    n_cols = 5
    n_rows = n_samples // n_cols if n_samples % n_cols == 0 else n_samples // n_cols + 1
    fig = plt.figure(figsize=(n_cols * 4, n_rows * 3))
    for i in range(n_samples):
        ax = fig.add_subplot(n_rows, n_cols, i + 1, xticks=[], yticks=[])
        ax.imshow(images[i])
        label = (
            labels[i].decode("utf-8")
            if label_names is None
            else label_names["c" + str(np.argmax(labels[i]))]
        )
        ax.set_title(f"Label: {label}")
        ax.axis("off")

In [ ]:
config = configparser.ConfigParser()
_ = config.read(os.path.join("..", "conf", "config.ini"))

project_name = config["project"]["project_name"]
run_name = config["project"]["run_name"]
raw_data_path = config["project"]["raw_data_path"]
validate_by_driver = eval(config["project"]["validate_by_driver"])
n_tfrec_chunks = eval(config["project"]["n_tfrec_chunks"])

model_url = config["model"]["model_url"]
fc_size = eval(config["model"]["fc_size"])
img_size = eval(config["model"]["img_size"])
n_epochs = eval(config["model"]["n_epochs"])
batch_size = eval(config["model"]["batch_size"])
initial_learning_rate = eval(config["model"]["initial_learning_rate"])
first_decay_steps = eval(config["model"]["first_decay_steps"])
use_adamw = eval(config["model"]["use_adamw"])
use_mixup = eval(config["model"]["use_mixup"])
use_swa = eval(config["model"]["use_swa"])
label_smoothing = eval(config["model"]["label_smoothing"])
n_train_splits = eval(config["model"]["n_train_splits"])
n_test_splits = eval(config["model"]["n_test_splits"])

label_names = {
    "c0": "safe driving",
    "c1": "texting - right",
    "c2": "talking on the phone - right",
    "c3": "texting - left",
    "c4": "talking on the phone - left",
    "c5": "operating the radio",
    "c6": "drinking",
    "c7": "reaching behind",
    "c8": "hair and makeup",
    "c9": "talking to passenger",
}

labels = list(label_names.keys())
num_classes = len(label_names)

In [ ]:
run = wandb.init(project=project_name, reinit=False)

if len(run_name) > 0:
    wandb.run.name = run_name
    wandb.run.save()
    
wandb.config = {
    "validate_by_driver": validate_by_driver,
    "n_tfrec_chunks": n_tfrec_chunks,
    "model_url": model_url,
    "fc_size": fc_size,
    "img_size": img_size,
    "n_epochs": n_epochs,
    "batch_size": batch_size,
    "initial_learning_rate": initial_learning_rate,
    "first_decay_steps": first_decay_steps,
    "use_adamw": use_adamw,
    "use_mixup": use_mixup,
    "use_swa": use_swa,
    "label_smoothing": label_smoothing,
    "n_train_splits": n_train_splits,
    "n_test_splits": n_test_splits,
}

## The Input Data Pipeline Configuration with *Dataset* API

In [ ]:
train_raw_data_path = os.path.join(raw_data_path, "imgs", "train")
test_raw_data_path = os.path.join(raw_data_path, "imgs", "test")

n_train_examples = len(
    tf.io.gfile.glob(os.path.join(train_raw_data_path, os.path.join("*", "*.jpg")))
)
test_ids = [
    os.path.basename(img_path)
    for img_path in np.sort(tf.io.gfile.glob(os.path.join(test_raw_data_path, "*.jpg")))
]
n_test_examples = len(test_ids)

In [ ]:
train_tfrec_paths = tf.io.gfile.glob(
    os.path.join(raw_data_path, "tfrec", "train", "*.tfrec")
)
test_tfrec_paths = np.sort(
    tf.io.gfile.glob(os.path.join(raw_data_path, "tfrec", "test", "*.tfrec"))
).tolist()

train_transforms = Compose(
    [
        ShiftScaleRotate(
            rotate_limit=(-20, 20),
            scale_limit=(0.0, 0.2),
            shift_limit_x=(-0.0625, 0.0625),
            shift_limit_y=(-0.046875, 0.046875),
            p=1.0,
        ),
    ]
)
valid_transforms = train_transforms

if n_test_splits > 1:
    test_transforms = train_transforms
else:
    def identity_func(image=None):
        return {"image": image}
    
    test_transforms = identity_func

n_samples = 20
sampled_dataset = load_tfrecord_dataset(
    [train_tfrec_paths[0]],
    (480, 640),
    train_transforms,
    n_samples,
    num_classes=num_classes,
)

view_image(sampled_dataset, n_samples, label_names)

## Building and Compiling the Model

In [ ]:
def get_model(verbose=False):
    model = tf.keras.Sequential(
        [
            hub.KerasLayer(
                model_url, trainable=False, input_shape=(img_size, img_size, 3)
            ),
            tf.keras.layers.Dense(fc_size, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(fc_size, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    if verbose:
        print(model.summary())

    # For tensorflow 2.5 or later, use tf.keras.optimizers.schedules.CosineDecayRestarts.
    lr_decayed_fn = tf.keras.experimental.CosineDecayRestarts(
        initial_learning_rate, first_decay_steps
    )
    optimizer = (
        tfa.optimizers.AdamW(lr_decayed_fn)
        if use_adamw
        else tfa.optimizers.RectifiedAdam(lr_decayed_fn)
    )
    if use_swa:
        optimizer = tfa.optimizers.SWA(optimizer, start_averaging=0, average_period=10)

    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing),
        metrics=["acc"],
    )
    
    return model

## Model Training and Prediction

In [ ]:
n = n_train_splits if n_train_splits > 1 else n_train_splits + 1
train_steps_per_epoch = round(
    n_train_examples * (n - 1) / n / batch_size
)
valid_steps_per_epoch = round(n_train_examples / n / batch_size)

test_dataset = load_tfrecord_dataset(
    test_tfrec_paths,
    img_size,
    test_transforms,
    1,
    shuffle=False,
    num_classes=num_classes,
    is_prediction=True,
)

if n_train_splits > 1:
    kf = KFold(n_splits=n_train_splits, shuffle=True, random_state=42)
    split = kf.split(range(len(train_tfrec_paths)))
else:
    rs = ShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
    split = rs.split(range(len(train_tfrec_paths)))

model_path = os.path.join("..", "models")
prediction_path = os.path.join("..", "predictions")

os.makedirs(model_path, exist_ok=True)
os.makedirs(prediction_path, exist_ok=True)

In [ ]:
for i, (train_index, valid_index) in enumerate(split):
    with get_elapsed_time():
        one_train_dataset = load_tfrecord_dataset(
            np.array(train_tfrec_paths)[train_index],
            img_size,
            train_transforms,
            batch_size,
            num_classes=num_classes,
        )

        if use_mixup:
            other_train_dataset = load_tfrecord_dataset(
                np.array(train_tfrec_paths)[train_index],
                img_size,
                train_transforms,
                batch_size,
                num_classes=num_classes,
            )
            zipped = tf.data.Dataset.zip((one_train_dataset, other_train_dataset))
            train_dataset = zipped.map(
                lambda x, y: mixup_dataset(x, y, alpha=0.2),
                num_parallel_calls=tf.data.experimental.AUTOTUNE,
            )
        else:
            train_dataset = one_train_dataset

        valid_dataset = load_tfrecord_dataset(
            np.array(train_tfrec_paths)[valid_index],
            img_size,
            valid_transforms,
            batch_size,
            num_classes=num_classes,
        )
        
        if n_train_splits > 1:
            suffix = "_" + str(i).zfill(2)
            print(
                f"Model training started on the {i + 1} of {n_train_splits} CV sets."
            )
        else:
            suffix = ""
            
        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor="val_loss", patience=5, restore_best_weights=True
            ),
            tf.keras.callbacks.ModelCheckpoint(
                os.path.join(model_path, f"model{suffix}.h5"),
                monitor="val_loss",
            ),
            WandbCallback(),
        ]

        model = get_model()
        model.fit(
            train_dataset,
            epochs=n_epochs,
            steps_per_epoch=train_steps_per_epoch,
            validation_data=valid_dataset,
            validation_steps=valid_steps_per_epoch,
            callbacks=callbacks,
        )

        for j in range(n_test_splits):
            if n_test_splits > 1:
                print(
                    f"Model prediction started with {j + 1} out of {n_test_splits} TTAs."
                )

            prediction = model.predict(
                test_dataset,
                steps=n_test_examples,
                use_multiprocessing=True,
            )

            if j == 0:
                predictions = prediction
            else:
                predictions += prediction

        predictions /= n_test_splits
        predictions = pd.DataFrame(predictions, columns=labels, index=test_ids)
        predictions.index.name = "img"
        predictions.to_csv(os.path.join(prediction_path, f"pred{suffix}.csv"))

        if i == 0:
            result = predictions
        else:
            result += predictions

result /= n_train_splits
result.to_csv(os.path.join(prediction_path, "submission.csv"))

print("The model training and prediction tasks have been successfully completed.")

In [ ]:
# lowerest
sampled_ids = np.random.choice(test_ids, n_samples)
columns = ["image_id", "image", "prediction"]

data = []
for each_id, predicted in zip(
    sampled_ids, predictions.loc[sampled_ids].values.argmax(axis=1)
):
    data.append(
        [
            each_id,
            wandb.Image(os.path.join(test_raw_data_path, each_id)),
            label_names[f"c{predicted}"],
        ]
    )

pred_img_table = wandb.Table(data=data, columns=columns)
wandb.log({"pred_img_table": pred_img_table})
run.finish()

In [ ]:
# Pseudo Label, GladCam, TF Serving
# SageMaker BiT, BYOS, Debugger